In [1]:
import IMAS
import IJulia

if true
    # if using Redis via docker
    REDIS_HOST = "localhost"
    REDIS_PORT = 55000
    REDIS_PASSWORD = "redispw"
else
    REDIS_HOST = "redis-19689.c281.us-east-1-2.ec2.cloud.redislabs.com"
    REDIS_PORT = 19689
    REDIS_PASSWORD = ""
end;

[ Info: Precompiling IMAS [13ead8c1-b7d1-41bb-a6d0-5b8b65ed587a]


In [3]:
function service_ip_control(stream_name; timeout=10.0)
    client = IMAS.Jedis.Client(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD);
    
    controller = IMAS.controllers__linear_controller()
    controller.name = "ip"
    
    while true
        # pop inputs
        inputs = IMAS.stream_pop!(client, "$(stream_name)__fuse2ctrl"; timeout, error_on_timeout=false)
        if inputs === nothing
            println("DONE $(stream_name)!")
            break
        end
        inputs = NamedTuple(inputs)

        # initialize controller with given PID coefficients
        if ismissing(controller.outputs, :data)
            IMAS.pid_controller(controller, inputs.P, inputs.I, inputs.D)
        end
        # run controller
        control = controller(inputs.setpoint, inputs.value, inputs.time)
        
        # push output
        IMAS.stream_push!(client, "$(stream_name)__ctrl2fuse"; control)
    end

    IMAS.Jedis.disconnect!(client)
end

service_ip_control (generic function with 1 method)

In [4]:
# automated spawn
subscription_client = IMAS.Jedis.Client(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD);
IMAS.stream_register_service("ip_control", service_ip_control; client=subscription_client)

In [3]:
# manual spawn
subscription_client = IMAS.Jedis.Client(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD);
IMAS.stream_register_service("ip_control", x->println(x); client=subscription_client)

7593161592688213358__ip


In [7]:
service_ip_control("7593161592688213358__ip")

DONE!
